In [4]:
from pickle import FALSE
from flask import Flask
from selenium import webdriver
import time
from datetime import datetime
import json
import pandas as pd
import os

In [2]:
def toJson(dir, dict):
    with open(dir, 'w', encoding='utf-8') as file:
        json.dump(dict, file, ensure_ascii=False, indent='\t')

def loadJson(dir):
    with open(dir, 'r', encoding='utf-8') as file:
        return json.load(file)

In [20]:
userId = 'musiclove529'
options = webdriver.ChromeOptions()
options.add_argument('--headleass')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)
driver.implicitly_wait(1)
url = "https://www.acmicpc.net/status?user_id={userId}&result_id=4".format(userId=userId)
sbDict = []
cnt = 1
driver.get(url)
while True:
    table = driver.find_element_by_tag_name('tbody')
    tr = table.find_elements_by_tag_name('tr')
    for i in tr:
        td = i.find_elements_by_tag_name('td')
        sbNum = td[0].text
        uId = td[1].text
        pId = td[2].text
        result = td[3].text
        memory = td[4].text
        time = td[5].text
        lang = td[6].text
        codeLen = td[7].text
        sbTime = td[8].find_element_by_tag_name('a').get_attribute('data-timestamp')
        sbDict.append({'sbNum':sbNum, 'uId':uId, 'pId':pId, 'result':result, 'memory':memory, 'time':time, 'lang':lang, 'codeLen':codeLen, 'sbTime':sbTime})
    try:
        nextPage = driver.find_element_by_id('next_page').get_attribute('href')
    except Exception:
        break
    driver.get(nextPage)
    print(cnt)
    cnt = cnt + 1
try:
    if not os.path.exists('submits/'):
        os.makedirs('submits/')
except OSError:
    print('Error : mkdir')
toJson('submits/' + userId + ".json", sbDict)
driver.quit()

1
2
3
4
5
6
7
8
9
10


In [46]:
categoryDF = pd.read_csv('category.csv', index_col=0, dtype=str, encoding='utf-8')
categoryDF

,pId,category
0,1002,4S
1,1003,19S
2,1004,12S
3,1005,44G
4,1007,31G
...,...,...
9568,24936,55S
9569,24937,3B
9570,24938,33S
9571,24939,55S


In [21]:
# 각 사용자 데이터에서 제출번호, 아이디, 문제 데이터만 추출하여 사용자가 푼 문제에 해당하는 태그 붙이기
file = 'submits/' + userId + '.json'
userDF = pd.read_json(file, dtype = str, encoding='utf-8')
userDF

,sbNum,uId,pId,result,memory,time,lang,codeLen,sbTime
0,43259988,musiclove529,1874,맞았습니다!!,50144,1656,Java 11,1410,1652449450
1,43252538,musiclove529,1253,맞았습니다!!,14848,184,Java 11,1166,1652438885
2,42319071,musiclove529,11660,맞았습니다!!,128856,1728,Java 11,1288,1650629701
3,42317293,musiclove529,11659,맞았습니다!!,59864,1468,Java 11,840,1650626871
4,41933378,musiclove529,1546,맞았습니다!!,18756,260,Java 11,480,1649852630
...,...,...,...,...,...,...,...,...,...
215,15501891,musiclove529,1000,맞았습니다!!,14248,108,Java 8,218,1570197725
216,15253710,musiclove529,10172,맞았습니다!!,12628,68,Java 8,254,1569079403
217,15253637,musiclove529,10171,맞았습니다!!,12708,76,Java 8,206,1569079083
218,15253362,musiclove529,10718,맞았습니다!!,12756,76,Java 8,173,1569078144


In [22]:
userDF = userDF[userDF['result']=='맞았습니다!!']
userDF

,sbNum,uId,pId,result,memory,time,lang,codeLen,sbTime
0,43259988,musiclove529,1874,맞았습니다!!,50144,1656,Java 11,1410,1652449450
1,43252538,musiclove529,1253,맞았습니다!!,14848,184,Java 11,1166,1652438885
2,42319071,musiclove529,11660,맞았습니다!!,128856,1728,Java 11,1288,1650629701
3,42317293,musiclove529,11659,맞았습니다!!,59864,1468,Java 11,840,1650626871
4,41933378,musiclove529,1546,맞았습니다!!,18756,260,Java 11,480,1649852630
...,...,...,...,...,...,...,...,...,...
215,15501891,musiclove529,1000,맞았습니다!!,14248,108,Java 8,218,1570197725
216,15253710,musiclove529,10172,맞았습니다!!,12628,68,Java 8,254,1569079403
217,15253637,musiclove529,10171,맞았습니다!!,12708,76,Java 8,206,1569079083
218,15253362,musiclove529,10718,맞았습니다!!,12756,76,Java 8,173,1569078144


In [23]:
userDF = userDF[userDF['pId'].isin(categoryDF['pId'])]
userDF

,sbNum,uId,pId,result,memory,time,lang,codeLen,sbTime
0,43259988,musiclove529,1874,맞았습니다!!,50144,1656,Java 11,1410,1652449450
1,43252538,musiclove529,1253,맞았습니다!!,14848,184,Java 11,1166,1652438885
2,42319071,musiclove529,11660,맞았습니다!!,128856,1728,Java 11,1288,1650629701
3,42317293,musiclove529,11659,맞았습니다!!,59864,1468,Java 11,840,1650626871
4,41933378,musiclove529,1546,맞았습니다!!,18756,260,Java 11,480,1649852630
...,...,...,...,...,...,...,...,...,...
205,15900766,musiclove529,2753,맞았습니다!!,14252,104,Java 8,285,1572436381
206,15502610,musiclove529,9498,맞았습니다!!,14276,104,Java 8,460,1570200172
207,15502482,musiclove529,1330,맞았습니다!!,14244,104,Java 8,324,1570199763
208,15502384,musiclove529,2588,맞았습니다!!,14240,116,Java 8,368,1570199439


In [24]:
userDF = userDF.sort_values(by='sbNum')
userDF

,sbNum,uId,pId,result,memory,time,lang,codeLen,sbTime
211,15502158,musiclove529,1008,맞았습니다!!,14500,108,Java 8,233,1570198629
208,15502384,musiclove529,2588,맞았습니다!!,14240,116,Java 8,368,1570199439
207,15502482,musiclove529,1330,맞았습니다!!,14244,104,Java 8,324,1570199763
206,15502610,musiclove529,9498,맞았습니다!!,14276,104,Java 8,460,1570200172
205,15900766,musiclove529,2753,맞았습니다!!,14252,104,Java 8,285,1572436381
...,...,...,...,...,...,...,...,...,...
4,41933378,musiclove529,1546,맞았습니다!!,18756,260,Java 11,480,1649852630
3,42317293,musiclove529,11659,맞았습니다!!,59864,1468,Java 11,840,1650626871
2,42319071,musiclove529,11660,맞았습니다!!,128856,1728,Java 11,1288,1650629701
1,43252538,musiclove529,1253,맞았습니다!!,14848,184,Java 11,1166,1652438885


In [25]:
if len(userDF) == 0: print()
sbTimes = []
userDF = userDF[['sbTime','uId','pId']]
df = pd.merge(userDF,categoryDF,how='left',on='pId')

# 라벨 붙이기
df['label'] = [1 for i in range(len(df))]

# '라벨', '아이디', '문제', '제출시간', '카테고리'순으로 열 바꾸기
df = df[['label', 'uId', 'pId', 'sbTime', 'category']]
df

,label,uId,pId,sbTime,category
0,1,musiclove529,1008,1570198629,41B
1,1,musiclove529,2588,1570199439,4B
2,1,musiclove529,1330,1570199763,41B
3,1,musiclove529,9498,1570200172,2B
4,1,musiclove529,2753,1572436381,55B
...,...,...,...,...,...
174,1,musiclove529,1546,1649852630,4B
175,1,musiclove529,11659,1650626871,16S
176,1,musiclove529,11660,1650629701,19S
177,1,musiclove529,1253,1652438885,40G


In [29]:
pHis = df['pId']
pStr = ""
for p in pHis:
    pStr = pStr + p + ","
pStr = pStr[:-1]
pStr

'1008,2588,1330,9498,2753,2884,10817,2739,10950,8393,2741,2742,11021,11022,2438,2439,10871,10952,10951,1110,10818,2562,2577,2743,5347,9656,5565,3052,1546,8958,2920,2440,2441,2442,2443,2444,17247,2445,10953,15596,11720,5597,4153,10569,10039,5543,10101,2480,11943,2525,5575,10707,5063,5596,2455,10797,5576,10817,2750,11650,5618,2858,5533,11004,11721,10250,10250,2309,2667,1012,2178,1075,2506,1920,15552,1920,17520,17520,2839,10814,10989,1188,11718,11718,11719,15719,3034,3047,2869,11367,2490,1427,15780,15725,10808,11131,1924,14655,2822,11109,11441,10809,12354,1181,10825,11651,15803,11772,10539,2751,1212,2108,1476,7326,10709,2563,2676,10872,10870,2675,4673,1316,1152,1157,2908,2675,1085,2231,2292,2292,2775,1978,2798,2798,1931,1931,1931,14500,2960,1475,1259,11050,10828,10845,1018,18352,1260,2012,11399,2693,7576,2805,10815,2512,1463,1149,1003,9095,1654,2606,1436,10866,1966,1966,2609,2252,1717,1806,11659,1644,2023,1697,1010,11720,1546,11659,11660,1253,1874'

In [30]:
cHis = df['category']
cStr = ""
for c in cHis:
    cStr = cStr + c + ","
cStr = cStr[:-1]
cStr

'41B,4B,41B,2B,55B,4B,2B,41B,41B,55B,2B,2B,41B,41B,2B,2B,55B,41B,41B,55B,55B,2B,41B,51B,65S,12S,41B,4B,4B,51B,2B,2B,2B,2B,2B,2B,4B,2B,7B,41B,7B,2B,4B,4B,4B,4B,2B,4B,55B,4B,41B,4B,4B,41B,41B,2B,54B,2B,54B,15S,31B,31B,41B,15S,51B,41B,41B,46B,24S,24S,1S,4B,41B,26S,41B,26S,4B,4B,48B,15S,15S,39G,51B,51B,51B,55B,4B,2B,4B,51B,2B,3S,4B,7S,51B,41B,2B,33B,15S,4B,16S,51B,11S,3S,15S,15S,4S,41B,4B,15S,32B,55S,31S,55B,2S,2B,4S,55B,55B,51B,55S,51S,51B,51B,55B,51B,4B,11B,4B,4B,55B,22S,11B,11B,15S,15S,15S,11G,22S,2S,51B,52B,13S,57S,11S,1S,24S,15S,15S,15S,1G,20S,20S,20S,19S,19S,19S,19S,20S,24S,11S,17S,57S,57S,65S,14G,0G,12G,16S,22G,50G,1S,48S,7B,4B,16S,19S,40G,13S'

In [31]:
tHis = df['sbTime']
tStr = ""
for t in tHis:
    tStr = tStr + t + ","
tStr = tStr[:-1]
tStr

'1570198629,1570199439,1570199763,1570200172,1572436381,1572437660,1572438312,1572511898,1572520733,1572521143,1572848863,1572849016,1572881440,1572881604,1572881833,1572882144,1572882820,1578639832,1578755044,1578756765,1580361768,1580362380,1580383031,1580384026,1580386296,1580390106,1580390775,1580526621,1581424311,1581426841,1581428592,1581489376,1581489994,1581490716,1581491308,1581503056,1581504920,1581777801,1581780896,1581781417,1583828728,1583829774,1583834951,1583836877,1583877736,1583878441,1583879176,1583880228,1583889451,1583892652,1583896450,1583899933,1583900411,1583901063,1583932599,1585065809,1586597011,1587189772,1587190835,1587192045,1587489631,1587493792,1587495452,1587499449,1587500648,1587633193,1587634578,1587643535,1589283422,1589285992,1590666617,1598946232,1598978974,1598981732,1600760048,1601969458,1602243232,1602243822,1602334768,1602338229,1602340129,1602349441,1602354569,1602354594,1602354679,1602764398,1604251461,1604253031,1604254215,1605787126,160578819

In [69]:
test_problem = categoryDF.copy()
test_problem

,pId,category
0,1002,4S
1,1003,19S
2,1004,12S
3,1005,44G
4,1007,31G
...,...,...
9568,24936,55S
9569,24937,3B
9570,24938,33S
9571,24939,55S


In [70]:
solved_problem = df['pId']
solved_problem

0       1008
1       2588
2       1330
3       9498
4       2753
       ...  
174     1546
175    11659
176    11660
177     1253
178     1874
Name: pId, Length: 179, dtype: object

In [71]:
remove_index = test_problem[test_problem['pId'].isin(solved_problem)].index
remove_index

Int64Index([   1,    5,    7,    9,   14,   56,   63,   82,  105,  107,
            ...
            5677, 6095, 6122, 6188, 6194, 6226, 6237, 6941, 7082, 7433],
           dtype='int64', length=165)

In [72]:
test_problem = test_problem.drop(remove_index)
test_problem

,pId,category
0,1002,4S
2,1004,12S
3,1005,44G
4,1007,31G
6,1009,55B
...,...,...
9568,24936,55S
9569,24937,3B
9570,24938,33S
9571,24939,55S


In [68]:
now = int(datetime.now().timestamp())
now

1652816212

In [75]:
index = test_problem.index
label = "0"
uId = "musiclove529"
now = str(int(datetime.now().timestamp()))
f = open('test_'+ uId,'w')
for i in index:
    data = test_problem.loc[i]
    pId = data['pId']
    cate = data['category']
    f.write(label + '\t' + uId + '\t' + pId + '\t' + cate + '\t' + now + '\t' + pStr + '\t' + cStr + '\t' + tStr + '\n')
